In [2]:
import os
import h5py
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold,StratifiedKFold, GroupKFold, KFold
import nilearn as nl
import torch
import random
from tqdm import tqdm
import itertools
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import PolynomialFeatures

import monai
from monai.transforms import \
    LoadNifti, LoadNiftid, AddChanneld, ScaleIntensityRanged, \
    Rand3DElasticd, RandAffined, \
    Spacingd, Orientationd

root = r'/home/youhanlee/project/trends/input/'
#root = r'/home/iclab/projects/Trends/input/'

train = pd.read_csv('{}/train_scores.csv'.format(root)).sort_values(by='Id')
loadings = pd.read_csv('{}/loading.csv'.format(root))
sample = pd.read_csv('{}/sample_submission.csv'.format(root))
reveal = pd.read_csv('{}/reveal_ID_site2.csv'.format(root))
ICN = pd.read_csv('{}/ICN_numbers.csv'.format(root))
fnc = pd.read_csv('{}/fnc.csv'.format(root))

In [3]:
fnc = pd.read_csv('{}/fnc.csv'.format(root))
fnc_columns = fnc.columns[1:]
fnc[fnc_columns] /= 500

degree = pd.read_csv('{}/degree.csv'.format(root))
degree_columns = degree.columns[:-1]
degree[degree_columns] /= 10000

In [4]:
labels_df = pd.read_csv("{}/train_scores.csv".format(root))
labels_df["is_train"] = True

In [6]:
labels_df.shape

(5877, 7)

In [21]:
features = ('age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2')
data = pd.merge(loadings, labels_df, on="Id", how="left")

good_feats = [f'IC_{feat}' for feat in ['15', '22', '06', '21', '04', '11', '10', '16']]
#data = pd.merge(loadings, train, on='Id').dropna()
comb = itertools.combinations(good_feats, 2)
for col1, col2 in comb:
    data[f'{col1}_x_{col2}'] = data[col1] * data[col2]
    data[f'{col1}_+_{col2}'] = data[col1] + data[col2]

id_test = list(data[data["is_train"] != True].Id)
fea_test = np.asarray(data.drop(list(features), axis=1).drop('Id', axis=1)[data["is_train"] != True].drop("is_train", axis=1))
lbl_test = np.asarray(data[list(features)][data["is_train"] != True])

data = pd.merge(fnc, labels_df, on="Id", how="left")
fnc_test = np.asarray(data.drop(list(features), axis=1).drop('Id', axis=1)[data["is_train"] != True].drop("is_train", axis=1))


In [22]:
fnc_test.shape

(5877, 1378)

In [23]:
data

,Id,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),...,CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4),age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,is_train
0,10001,0.000737,0.000334,0.000876,0.000682,-0.000373,0.000098,0.000243,-0.000349,-0.000463,...,0.000249,-0.000121,0.001032,0.000581,57.436077,30.571975,62.553736,53.325130,51.427998,True
1,10002,0.000303,-0.000050,0.000435,0.000836,-0.000454,-0.000128,-0.000288,-0.000236,-0.000110,...,0.001185,0.000057,0.001411,0.000497,59.580851,50.969456,67.470628,60.651856,58.311361,True
2,10003,0.000687,0.000220,0.001483,0.001157,-0.001353,-0.000874,-0.000591,-0.000756,-0.000690,...,0.001128,0.000877,0.001236,0.000569,NaN,NaN,NaN,NaN,NaN,NaN
3,10004,0.000266,0.000517,0.000982,0.000685,0.000182,0.000216,0.000058,-0.000052,0.000189,...,0.000635,0.000025,0.001332,0.000163,71.413018,53.152498,58.012103,52.418389,62.536641,True
4,10005,0.000584,0.000503,0.000833,0.001023,-0.000725,-0.000329,-0.000578,-0.000031,-0.000175,...,0.000455,0.000058,0.001505,0.000176,66.532630,NaN,NaN,52.108977,69.993075,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.000456,-0.000445,0.000501,0.000885,-0.000442,-0.000470,-0.000593,-0.000546,-0.000213,...,0.000916,0.000688,0.001410,0.000682,48.948756,55.114811,60.878271,38.617246,50.679885,True
11750,21751,0.000910,0.000968,0.001179,0.001267,0.000324,-0.000351,0.000031,0.000271,0.000345,...,0.001030,0.000759,0.001137,0.000878,NaN,NaN,NaN,NaN,NaN,NaN
11751,21752,0.000237,0.000904,0.001217,0.000845,-0.000213,-0.000017,-0.000020,-0.000244,0.000186,...,0.000664,0.000458,0.001122,0.000527,66.532630,59.844808,72.303110,55.458281,46.870235,True
11752,21753,0.000102,0.000177,0.001103,0.000611,-0.000069,-0.000190,-0.000180,-0.000446,-0.000395,...,0.000462,0.000335,0.000443,0.000743,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
data = pd.merge(degree, labels_df, on='Id', how='left')

In [25]:
data

,degree_2,degree_3,degree_4,degree_5,degree_7,degree_8,degree_9,degree_11,degree_12,degree_13,...,degree_96,degree_98,degree_99,Id,age,domain1_var1,domain1_var2,domain2_var1,domain2_var2,is_train
0,0.000070,-0.000046,-0.000217,-1.184378e-04,-0.000197,-0.000091,9.605052e-05,0.000004,-0.000048,0.000125,...,-0.000152,-0.000071,-0.000038,10001,57.436077,30.571975,62.553736,53.325130,51.427998,True
1,-0.000257,-0.000009,-0.000175,1.994855e-04,0.000084,0.000134,-7.200379e-05,-0.000143,0.000240,0.000062,...,-0.000320,0.000057,0.000134,10002,59.580851,50.969456,67.470628,60.651856,58.311361,True
2,-0.000176,-0.000103,-0.000168,8.061461e-05,-0.000124,0.000072,-6.163292e-05,-0.000134,-0.000018,-0.000008,...,-0.000165,-0.000025,-0.000228,10003,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.000057,0.000065,-0.000122,-2.274933e-05,0.000014,0.000042,4.879794e-05,0.000059,0.000034,-0.000015,...,-0.000196,-0.000017,0.000028,10004,71.413018,53.152498,58.012103,52.418389,62.536641,True
4,-0.000093,0.000007,-0.000183,-1.381382e-04,-0.000152,-0.000152,-6.640828e-05,0.000056,-0.000135,-0.000152,...,-0.000030,-0.000069,0.000011,10005,66.532630,NaN,NaN,52.108977,69.993075,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,-0.000151,-0.000253,-0.000120,-1.167254e-04,-0.000152,-0.000071,-2.010326e-04,-0.000251,-0.000012,0.000047,...,-0.000140,-0.000111,0.000064,21750,48.948756,55.114811,60.878271,38.617246,50.679885,True
11750,-0.000016,0.000004,-0.000071,-1.457717e-04,-0.000123,-0.000178,3.613260e-07,0.000009,-0.000071,-0.000112,...,-0.000180,-0.000002,0.000059,21751,NaN,NaN,NaN,NaN,NaN,NaN
11751,-0.000197,-0.000074,0.000014,-8.085483e-06,-0.000005,0.000059,-2.001639e-04,-0.000095,-0.000279,0.000103,...,0.000067,0.000080,0.000145,21752,66.532630,59.844808,72.303110,55.458281,46.870235,True
11752,0.000109,-0.000020,-0.000242,1.469520e-07,-0.000326,0.000134,4.463215e-05,0.000057,0.000131,-0.000140,...,-0.000281,0.000004,-0.000018,21753,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
(degree['Id'] == fnc['Id']).sum(0)

11754

In [17]:
degree.shape

(11754, 54)

In [29]:
good_feats = [f'IC_{feat}' for feat in ['15', '22', '06', '21', '04', '11', '10', '16']]

In [30]:
fnc = pd.read_csv('{}/fnc.csv'.format(root))
features = ('age', 'domain1_var1', 'domain1_var2', 'domain2_var1', 'domain2_var2')
data = pd.merge(loadings, train, on='Id').dropna()

In [31]:
comb = itertools.combinations(good_feats, 2)
for col1, col2 in comb:
    data[f'{col1}_x_{col2}'] = data[col1] * data[col2]
    data[f'{col1}_+_{col2}'] = data[col1] + data[col2]

In [32]:
id_train = list(data.Id)
fea_train = np.asarray(data.drop(list(features), axis=1).drop('Id', axis=1))
lbl_train = np.asarray(data[list(features)])

In [33]:
fea_train.shape

(5434, 82)